In [2]:
import src.data.bigearthnet_datapipes as ben_pipes
import src.data.general_datapipes as pipes
import fsspec
import json

This file creates a pickle file containing a dict of labels for the bigerathnet dataset. The file is in the repo too and is red by globals.LABELS_TO_INDS, so you should not need to run this here. 

In [3]:
file_list = list(pipes.get_s3_folder_content())

In [9]:
import torchdata.datapipes as dp
import tqdm
import pickle

pipe = dp.iter.IterableWrapper(file_list[:50000])
pipe = pipe.list_files_by_fsspec()
pipe = pipe.groupby(group_key_fn=ben_pipes.group_key_by_folder, group_size=13)
pipe = pipe.map(ben_pipes.chunk_to_dataloader_dict)
pipe = pipe.map(lambda x: x["label"])
pipe = pipe.map(lambda x: json.loads(fsspec.open(x, mode="r").open().read())["labels"])

classes = list()
for js in tqdm.tqdm(pipe):
    for label in js:
        classes.append(label)

from collections import Counter

counter = Counter(classes)

classes = sorted(set(classes))

classes_to_ind = {i: cls for (i, cls) in enumerate(classes)}

print(counter)
print(classes)
print(classes_to_ind)

with open("/home/pfuerste/Projects/MI4People_soil_quality/data/raw/ben_labels", "wb") as p_out:
    pickle.dump(classes_to_ind, p_out)

50000it [2:14:23,  6.20it/s]


Counter({'Non-irrigated arable land': 17032, 'Mixed forest': 15176, 'Coniferous forest': 14565, 'Pastures': 14279, 'Transitional woodland/shrub': 11723, 'Broad-leaved forest': 9197, 'Land principally occupied by agriculture, with significant areas of natural vegetation': 9085, 'Sea and ocean': 8707, 'Complex cultivation patterns': 6348, 'Discontinuous urban fabric': 5999, 'Water bodies': 4428, 'Agro-forestry areas': 3123, 'Permanently irrigated land': 2052, 'Peatbogs': 1945, 'Olive groves': 1704, 'Industrial or commercial units': 1390, 'Water courses': 1210, 'Moors and heathland': 1171, 'Vineyards': 832, 'Sport and leisure facilities': 798, 'Annual crops associated with permanent crops': 779, 'Rice fields': 741, 'Inland marshes': 597, 'Sclerophyllous vegetation': 554, 'Road and rail networks and associated land': 532, 'Mineral extraction sites': 506, 'Natural grassland': 443, 'Continuous urban fabric': 332, 'Estuaries': 280, 'Beaches, dunes, sands': 270, 'Intertidal flats': 243, 'Salt 